In [1]:
def day_to_week(df,isrolling=True,rollinig_period = 20,keywords='CLOSE'):
    """
    周度数据，高频变低频，均值-前值填充
    """
    if isrolling==True:
        df['rolling'] = df[keywords].rolling(rollinig_period).mean() 
    else:
        df = df
    df.index = pd.to_datetime(df.index)
    df_ = df.resample('W-SUN').last().ffill().dropna()  # resample用前最邻近值
    return df_

In [2]:
def month_to_week(df):
    """
    down resample 由于直接resample会导致数据确实，需要先根据已有index，使用date_range创建新的index；
    其中，pd.offsets.monthbegin和pd.offsets.monthend用来获得月初月末日期
    """
    first_date = inx2.index.min()  # 第一天
    start_of_month = first_date - pd.offsets.MonthBegin(1)  # 月初
    if first_date != start_of_month:
        df = df.reindex(pd.date_range(start=start_of_month, end=df.index.max(), freq='W-MON'))  # 用新的datetimeindex作index
        df.bfill(inplace=True)
    return df

In [3]:
def get_roc_idc(df,timeperiod=10):
    df['roc'] = ta.ROC(df['CLOSE'].values, timeperiod)
    return df['roc']

In [4]:
def get_sma_idc(df, timeperiod=20):
    df['sma'] = ta.SMA(df['CLOSE'].values, timeperiod)
    return df['sma']

In [5]:
def get_dma_idc(df,short_tp=10,long_tp=20):
    short_ma = ta.SMA(df['CLOSE'].values, short_tp)
    long_ma = ta.SMA(df['CLOSE'].values, long_tp)
    df['dma'] = short_ma - long_ma
    return df['dma']

In [6]:
def get_ama_idc(df,short_tp=10,long_tp=50):
    short_ma = ta.SMA(df['CLOSE'].values, short_tp)
    long_ma = ta.SMA(df['CLOSE'].values, long_tp)
    df['dma'] = short_ma - long_ma
    df['ama'] = df['dma'].rolling(10).mean()
    return df['ama']

In [7]:
def get_macd_idc(df,fastperiod=12, slowperiod=26, signalperiod=9):
    df['macd'] = ta.MACD(df['CLOSE'].values, fastperiod,slowperiod,signalperiod)[0]  # DEA是signal,ta.MACD返回三个值
    return df['macd']

In [8]:
def get_trix_idc(df,timeperiod=12,signalperiod=9):
    trix = pd.Series(ta.TRIX(df['CLOSE'].values, timeperiod))
    matrix = trix.rolling(signalperiod).mean()  # rolling 包含当日
    df['trix_diff'] = (trix - matrix).values  # series to df 要用pd.调用方法，nparray可以直接放进去
    return df['trix_diff']

In [9]:
def get_bbi_idc(df,tp1=3,tp2=6,tp3=12,tp4=24):
    bbi_n1 = ta.SMA(df['CLOSE'].values, timeperiod=tp1)
    bbi_n2 = ta.SMA(df['CLOSE'].values, timeperiod=tp2)
    bbi_n3 = ta.SMA(df['CLOSE'].values, timeperiod=tp3)
    bbi_n4 = ta.SMA(df['CLOSE'].values, timeperiod=tp4)
    df['bbi'] = (bbi_n1 + bbi_n2 + bbi_n3 + bbi_n4) / 4
    return df['bbi']

In [10]:
# bbands和其他指标获取区别在于，bbands保留了close数据，因为在后续获得信号时，bbands需要close
def get_bbands_idc(df,timeperiod=20, nbdevup=2, nbdevdn=2):
    df['upperband'], middleband, df['lowerband'] = ta.BBANDS(df['CLOSE'].values, timeperiod, nbdevup, nbdevdn, matype=0)
    return df[['CLOSE','upperband','lowerband']]

In [11]:
def get_bias_idc(df,timeperiod=10):
    sma = ta.SMA(df['CLOSE'].values, timeperiod)
    df['bias'] = (df['CLOSE'] - sma) / sma * 100
    return df['bias']

In [12]:
def get_cci_idc(df,timeperiod=14):
    df['cci'] = ta.CCI(df['CLOSE'].values, df['CLOSE'].values, df['CLOSE'].values, timeperiod)
    return df['cci']

In [13]:
def get_kdj_idc(df,fastk_period=9, slowk_period=3, slowd_period=3):
    
    def sma_csdn(close, timeperiod):
        close = np.nan_to_num(close)
        return reduce(lambda x, y: ((timeperiod - 1) * x + y) / timeperiod, close, timeperiod)

    def stock_csdn(high, low, close, fastk_period, slowk_period, slowd_period):
        kValue, dValue = ta.STOCHF(high, low, close, fastk_period, fastd_period=1, fastd_matype=0)

        kValue = np.array(list(map(lambda x: sma_csdn(kValue[:x], slowk_period), range(1, len(kValue) + 1))))
        dValue = np.array(list(map(lambda x: sma_csdn(kValue[:x], slowd_period), range(1, len(dValue) + 1))))
        jValue = 3 * kValue - 2 * dValue

        func = lambda arr: np.array([0 if x < 0 else (100 if x > 100 else x) for x in arr])

        kValue = func(kValue)
        dValue = func(dValue)
        jValue = func(jValue)
        return kValue, dValue
    
    # 通过STOCK获取K和D
    df['slowk'], df['slowd'] = stock_csdn(df['HIGH'].values, df['LOW'].values, df['CLOSE'].values, fastk_period, slowk_period, slowd_period)
    # 获取J，这里没有用到
    list_slowj= list(map(lambda x, y: 3 * x - 2 * y, df['slowk'], df['slowd']))
    return df[['slowk','slowd']]

In [14]:
def get_rsi_idc(df, timeperiod=6):
    df['rsi'] = ta.RSI(df['CLOSE'], timeperiod=6)
    return df['rsi']

In [15]:
def get_cmo_idc(df,timeperiod=12):
    df['cmo'] = ta.CMO(df['CLOSE'], timeperiod)
    return df['cmo']

In [16]:
import pandas as pd
import numpy as np

import os
from WindPy import w
import datetime
import talib as ta
from functools import reduce
from sqlalchemy import create_engine



_ = w.start()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


# MetaData Update

In [45]:
start_date='2024-06-10'  # 周一
start_date_='2024-06-07'  # start_date前一个工作日，用于提取wind全A时收益率计算
start_date_monthly = '2024-04-30'  # 月频数据，需要上上个月月末日期，只有一个月末没有日期序列
end_date='2024-06-16'  # 周日

In [18]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/meta_data?charset=utf8mb4")

In [59]:
df_881001 = pd.read_sql('SELECT Date,HIGH,LOW,CLOSE FROM 881001_wi', engine,index_col='Date')  # 从meta_data提取数据
df_881001_ = day_to_week(df_881001,isrolling=False)

## 宏观因子

In [10]:
print('***'*33)

***************************************************************************************************


In [19]:
error_code, inx1 = w.edb("M0041653",start_date,end_date,"Fill=Previous", usedf=True)
inx1.to_sql(name="zyhgitt",con=engine,if_exists='append',index=True,index_label='Date')

4

In [34]:
# 需要两个月，然后drop duplicate，否则没有日期
error_code, inx2 = w.edb("M0043418", start_date_monthly,end_date, "Fill=Previous", usedf=True)
inx2.to_sql(name="zcqlb",con=engine,if_exists='append',index=True,index_label='Date')

2

In [35]:
error_code, inx3 = w.edb("M0017129", start_date_monthly,end_date, "Fill=Previous", usedf=True)
inx3.to_sql(name="xckpmiidy",con=engine,if_exists='append',index=True,index_label='Date')

2

In [36]:
path = os.path.join(os.getcwd(), 'db')
inx4 = pd.read_excel('{}/中国社会融资规模存量同比.xlsx'.format(path), index_col=0, parse_dates=True,names=['CLOSE'])
inx4.to_sql(name="srclyoy",con=engine,if_exists='replace',index=True,index_label='Date')

233

In [37]:
error_code, inx5 = w.edb("S0059749,G0000891",start_date,end_date, "Fill=Previous", usedf=True)  # sq：中国，美国
inx5.to_sql(name="tb10y",con=engine,if_exists='append',index=True,index_label='Date')

5

In [38]:
error_code, inx6 = w.edb("M0060433", start_date,end_date, "Fill=Previous", usedf=True)
inx6.to_sql(name="nrfs",con=engine,if_exists='append',index=True,index_label='Date')

3

In [39]:
error_code, inx7 = w.edb("M1000166,M0330161",start_date,end_date, "Fill=Previous", usedf=True)  # sq：中债国债，TTM
inx7.to_sql(name="tb10y_ttm",con=engine,if_exists='append',index=True,index_label='Date')

7

## 技术指标

In [9]:
print('***'*33)

***************************************************************************************************


In [42]:
error_code, nv_index_e = w.wsd("881001.WI", "open,high,low,close", start_date_,end_date, usedf=True)
nv_index_e['ret'] = nv_index_e['CLOSE'].pct_change(1)
nv_index_e
nv_index_e.to_sql(name='881001_wi',con=engine,if_exists='append',index=True,index_label='Date')

5

In [43]:
erro_code, nv_index_b = w.wsd("CBA00101.CS", "close",start_date_,end_date, usedf=True)
nv_index_b['ret'] = nv_index_b['CLOSE'].pct_change(1)
nv_index_b
nv_index_b.to_sql(name='cba00101_cs',con=engine,if_exists='append',index=True,index_label='Date')

5

## 情绪指标

In [15]:
print('***'*33)

***************************************************************************************************


In [44]:
errorcode, rqmr_df = w.edb("M0075987",start_date,end_date, "Days=Alldays;Fill=Previous", usedf=True)  # start_date is 2010-03-31
rqmr_df.to_sql(name='rzmr',con=engine,if_exists='append',index=True,index_label='Date')

7

In [46]:
errorcode, ic_cfe = w.wsd("IC.CFE", "anal_basisannualyield",start_date,end_date, "Days=Alldays;Fill=Previous",usedf=True) 
ic_cfe.to_sql(name='ic_cfe',con=engine,if_exists='append',index=True,index_label='Date')

7

In [47]:
errorcode, if_cfe = w.wsd("IF.CFE", "anal_basisannualyield",start_date,end_date, "Days=Alldays;Fill=Previous",usedf=True)  
if_cfe.to_sql(name='if_cfe',con=engine,if_exists='append',index=True,index_label='Date')

7

In [48]:
errorcode, vol_df = w.wsd("510050.SH", "putvolume,callvolume", start_date,end_date,"unit=1;Days=Alldays;Fill=Previous",usedf=True)  
vol_df.to_sql(name='vol',con=engine,if_exists='append',index=True,index_label='Date')

7

In [49]:
errorcode, oi_df = w.wsd("510050.SH", "calloi,putoi",start_date,end_date, "unit=1;Days=Alldays;Fill=Previous",usedf=True) 
oi_df.to_sql(name='oi',con=engine,if_exists='append',index=True,index_label='Date')

7

# FactorPool_Update

- 如果是每周一更新，那数据正好截至到上周日，不需要额外处理

In [89]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/meta_data?charset=utf8mb4")

In [91]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/factors_pool?charset=utf8mb4")

#### 银行间质押式回购加权利率:7天

In [51]:
inx1 = pd.read_sql('SELECT * FROM zyhgitt', engine,index_col='Date')  # 从meta_data提取数据
inx1_ = pd.DataFrame(day_to_week(inx1).loc[:,'rolling'])  # 获取因子值
inx1_.rename(columns={'rolling':'CLOSE'},inplace=True)

In [84]:
inx1_.to_sql(name='zyhgitt_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1324

#### 中长期贷款余额:同比

In [52]:
inx2 = pd.read_sql('SELECT * FROM zcqlb',engine,index_col='Date')
inx2['CLOSE'] = (inx2['CLOSE'] - inx2['CLOSE'].shift(12)) / inx2['CLOSE'].shift(12) * 100

In [85]:
inx2.to_sql(name='zcqlb_m',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

365

#### 制造业PMI:新出口订单

In [53]:
inx3 = pd.read_sql('SELECT * FROM xckpmiidy', engine,index_col='Date')

In [86]:
inx3.to_sql(name='xckpmiidy_m',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

233

#### 社会融资规模存量:同比

In [54]:
inx4 = pd.read_sql('SELECT * FROM srclyoy', engine,index_col='Date')

In [87]:
inx4.to_sql(name='srclyoy_m',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

233

#### 中美十年国债收益率_差值

In [90]:
inx5 = pd.read_sql('SELECT * FROM tb10y', engine,index_col='Date')
inx5['DIFF'] = inx5['S0059749'] - inx5['G0000891']
inx5_ = pd.DataFrame(day_to_week(inx5,keywords='DIFF').loc[:,'rolling'])
inx5_.rename(columns={'rolling':'CLOSE'},inplace=True)

In [92]:
inx5_.to_sql(name='tb10y_diff_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1168

#### 新成立基金份额

In [56]:
inx6 = pd.read_sql('SELECT * FROM nrfs', engine,index_col='Date')  # 日频数据，但20050101之前数据质量差，已清洗
inx6_ = pd.DataFrame(day_to_week(inx6).loc[:,'rolling'])
inx6_.rename(columns={'rolling':'CLOSE'},inplace=True)

In [93]:
inx6_.to_sql(name='nrfs_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1205

#### 股债性价比

In [57]:
inx7 = pd.read_sql('SELECT * FROM tb10y_ttm', engine,index_col='Date')
inx7['DIFF'] = inx7['M1000166'] - 1 / inx7['M0330161']
inx7_ = pd.DataFrame(day_to_week(inx7,keywords='DIFF').loc[:,'rolling'])
inx7_.rename(columns={'rolling':'CLOSE'},inplace=True)

In [94]:
inx7_.to_sql(name='tb10ymttm_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1169

#### roc

In [61]:
roc_idc = get_roc_idc(df_881001_)

In [95]:
roc_idc.to_sql(name='roc_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### sma

In [62]:
sma_idc = get_sma_idc(df_881001_)

In [96]:
sma_idc.to_sql(name='sma_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### dma

In [63]:
dma_idc = get_dma_idc(df_881001_)

In [97]:
dma_idc.to_sql(name='dma_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### ama

In [64]:
ama_idc = get_ama_idc(df_881001_)

In [98]:
ama_idc.to_sql(name='ama_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### macd

In [65]:
macd_idc = get_macd_idc(df_881001_)

In [99]:
macd_idc.to_sql(name='macd_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### trix

In [66]:
trix_idc = get_trix_idc(df_881001_)

In [100]:
trix_idc.to_sql(name='trix_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### bbi

In [67]:
bbi_idc = get_bbi_idc(df_881001_)

In [101]:
bbi_idc.to_sql(name='bbi_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### bbands

In [68]:
bbands_idc = get_bbands_idc(df_881001_)

In [102]:
bbands_idc.to_sql(name='bbands_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### bias

In [69]:
biac_idc = get_bias_idc(df_881001_)

In [103]:
biac_idc.to_sql(name='bias_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### cci

In [70]:
cci_idc = get_cci_idc(df_881001_)

In [104]:
cci_idc.to_sql(name='cci_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### KDJ

In [71]:
kd_idc = get_kdj_idc(df_881001_)

In [105]:
kd_idc.to_sql(name='kdj_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### rsi

In [72]:
rsi_idc = get_rsi_idc(df_881001_)

In [106]:
rsi_idc.to_sql(name='rsi_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### cmo

In [73]:
cmo_idc = get_cmo_idc(df_881001_)

In [107]:
cmo_idc.to_sql(name='cmo_881001_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

#### rzmr

In [74]:
df_rzmr = pd.read_sql('SELECT * FROM rzmr', engine,index_col='Date')  # 从meta_data提取数据
df_rzmr_ = day_to_week(df_rzmr,isrolling=True)
df_rzmr_['CLOSE'] = df_rzmr_['rolling']-df_rzmr_['rolling'].shift(1)  # rzmr数据处理

In [75]:
bbands_idc_rzmr = get_bbands_idc(df_rzmr_,nbdevup=1.5, nbdevdn=1.5)  # 获取indicator

In [108]:
bbands_idc_rzmr.to_sql(name='bbands_rzmr_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

739

#### ic_cfe

In [76]:
df_ic_cfe = pd.read_sql('SELECT * FROM ic_cfe', engine,index_col='Date')  # 从meta_data提取数据
df_ic_cfe.rename(columns={'ANAL_BASISANNUALYIELD':'CLOSE'},inplace=True)
df_ic_cfe_ = day_to_week(df_ic_cfe,isrolling=True)  
df_ic_cfe_['CLOSE'] = df_ic_cfe_['rolling']

In [77]:
bbands_idc_iccfe = get_bbands_idc(df_ic_cfe_,nbdevup=1.5, nbdevdn=1.5)  # 获取indicator

In [109]:
bbands_idc_iccfe.to_sql(name='bbands_iccfe_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

476

#### if_cfe

In [78]:
df_if_cfe = pd.read_sql('SELECT * FROM if_cfe', engine,index_col='Date')  # 从meta_data提取数据
df_if_cfe.rename(columns={'ANAL_BASISANNUALYIELD':'CLOSE'},inplace=True)
df_if_cfe_ = day_to_week(df_if_cfe,isrolling=True)  
df_if_cfe_['CLOSE'] = df_if_cfe_['rolling']

In [79]:
bbands_idc_ifcfe = get_bbands_idc(df_if_cfe_,nbdevup=1.5, nbdevdn=1.5)  # 获取indicator

In [110]:
bbands_idc_ifcfe.to_sql(name='bbands_ifcfe_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

737

#### vol

In [80]:
df_vol = pd.read_sql('SELECT * FROM vol', engine,index_col='Date')  # 从meta_data提取数据
df_vol['volumeratio'] = df_vol['CALLVOLUME'] / df_vol['PUTVOLUME']  # 计算1/PCR
df_vol_ = day_to_week(df_vol,keywords='volumeratio')
df_vol_['volumeratio_'] = (df_vol_['rolling'] - df_vol_['rolling'].shift(1)).rolling(60).mean()

In [111]:
df_vol_['volumeratio_'].to_sql(name='volumeratio_w',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

486

#### oi

需要注意的，若当月没过完，则不能有当月的月度平均值

In [81]:
df_oi = pd.read_sql('SELECT * FROM oi', engine,index_col='Date')  # 从meta_data提取数据
df_oi['oiratio'] = (df_oi['PUTOI'] / df_oi['CALLOI']).rolling(20).mean()

In [82]:
# 用到月频策略，变成月频数据
df_oi.index = pd.to_datetime(df_oi.index)
df_oi_ = df_oi.resample('M').mean()

In [112]:
df_oi_['oiratio'].to_sql(name='oiratio_m',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

113